In [437]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tqdm import tqdm
from time import time

In [438]:
np.random.seed(1234)
tf.set_random_seed(1234)
IMAGE_HEIGHT = 28
IMAGE_WIDTH  = 28
batch_size = 20
NUM_CLASSES = 10
l2_lamb = 1e-5

In [439]:
def reshape(x):
    return np.reshape(x, [-1,IMAGE_WIDTH,IMAGE_HEIGHT,1])

In [440]:
def get_mnist():
    datum = {}
    mnist = tf.keras.datasets.mnist
    (x_train, y_train),(x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    randos = np.random.choice(60000,60000)
    validation_randos = randos[:12000]
    train_randos = randos[12000:]
    val_x, val_y = reshape(x_train[validation_randos]), y_train[validation_randos]
    x_train, y_train = reshape(x_train[train_randos]), y_train[train_randos]
    x_test = reshape(x_test)
    y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
    y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)
    val_y = keras.utils.to_categorical(val_y,NUM_CLASSES)
    datum = {"x_train":x_train, "y_train":y_train, "x_val":val_x, "y_val":val_y,"x_test":x_test,"y_test":y_test}
    return datum

In [441]:
datum = get_mnist()

In [499]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(filters=2,kernel_size=(4,4),strides=2,padding='valid',activation='elu'))
model.add(keras.layers.MaxPool2D(pool_size=(3,3)))
model.add(keras.layers.Conv2D(filters=1,kernel_size=(2,2),strides=1,padding='valid',activation='elu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(NUM_CLASSES,activation="softmax"))
model.compile(optimizer=tf.train.AdamOptimizer(0.01),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [500]:
tensorboard = tf.keras.callbacks.TensorBoard(log_dir="logs/{}".format(time()))
model.fit(datum["x_train"],datum["y_train"], epochs=200, batch_size=256,
          validation_data=(datum["x_val"], datum["y_val"]), verbose=2, callbacks=[tensorboard])

Train on 48000 samples, validate on 12000 samples
Epoch 1/200
 - 8s - loss: 1.4776 - acc: 0.4686 - val_loss: 1.0665 - val_acc: 0.6252
Epoch 2/200
 - 3s - loss: 1.0056 - acc: 0.6496 - val_loss: 0.9495 - val_acc: 0.6772
Epoch 3/200
 - 3s - loss: 0.9287 - acc: 0.6853 - val_loss: 0.8902 - val_acc: 0.7013
Epoch 4/200
 - 3s - loss: 0.8945 - acc: 0.7003 - val_loss: 0.8711 - val_acc: 0.7080
Epoch 5/200
 - 4s - loss: 0.8773 - acc: 0.7078 - val_loss: 0.8519 - val_acc: 0.7147
Epoch 6/200
 - 3s - loss: 0.8670 - acc: 0.7124 - val_loss: 0.8482 - val_acc: 0.7232
Epoch 7/200
 - 3s - loss: 0.8593 - acc: 0.7191 - val_loss: 0.8438 - val_acc: 0.7219
Epoch 8/200
 - 3s - loss: 0.8488 - acc: 0.7259 - val_loss: 0.8222 - val_acc: 0.7344
Epoch 9/200
 - 3s - loss: 0.8382 - acc: 0.7311 - val_loss: 0.8089 - val_acc: 0.7407
Epoch 10/200
 - 3s - loss: 0.8254 - acc: 0.7367 - val_loss: 0.7987 - val_acc: 0.7437
Epoch 11/200
 - 3s - loss: 0.8083 - acc: 0.7436 - val_loss: 0.7854 - val_acc: 0.7483
Epoch 12/200
 - 3s - los

KeyboardInterrupt: 

In [501]:
print(model.count_params())
for layer in model.layers:
    print(layer.get_output_at(0).get_shape().as_list())

143
[None, 13, 13, 2]
[None, 4, 4, 2]
[None, 3, 3, 1]
[None, 9]
[None, 10]


In [502]:
model.evaluate(datum["x_test"],datum["y_test"])

10000/10000 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 2s 202us/step


[0.45329682639837265, 0.8576]